In [4]:
import torch 
from dgl.data.utils import load_graphs


In [11]:
sent_state = torch.load('sent_state')
result = torch.load("result")

In [6]:
graphs,_ = load_graphs("graphs.bin")
graph = graphs[0]

In [7]:
graph

Graph(num_nodes=14316, num_edges=176738,
      ndata_schemes={'unit': Scheme(shape=(), dtype=torch.float32), 'id': Scheme(shape=(), dtype=torch.int64), 'dtype': Scheme(shape=(), dtype=torch.float32), 'words': Scheme(shape=(100,), dtype=torch.int64), 'position': Scheme(shape=(1,), dtype=torch.int64), 'sent_embedding': Scheme(shape=(300,), dtype=torch.float32), 'label': Scheme(shape=(50,), dtype=torch.int64), 'embed': Scheme(shape=(300,), dtype=torch.float32)}
      edata_schemes={'tffrac': Scheme(shape=(), dtype=torch.int64), 'dtype': Scheme(shape=(), dtype=torch.float32), 'tfidfembed': Scheme(shape=(50,), dtype=torch.float32), 'e': Scheme(shape=(1,), dtype=torch.float32)})

In [18]:
torch.argmax(result,axis=1).sum()

tensor(13, device='cuda:0')

In [20]:
sent_state.shape

torch.Size([1952, 64])

In [23]:
result

tensor([[ 0.3698, -0.6745],
        [ 0.3115, -0.6124],
        [ 1.1771, -1.3010],
        ...,
        [ 1.4368, -1.6777],
        [ 1.4608, -2.0875],
        [ 0.1937, -0.1417]], device='cuda:0')

In [51]:
class SentenceLevelModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.LSTM(64, 20, 1,bias=True)
        self.classifier = torch.nn.Linear(20,2)
    
    def forward(self,x,probabilities):
        x = self.rnn(x)[0]
        out = self.classifier(x)
        return out+probabilities/2

In [52]:
device = torch.device("cuda:0")

In [53]:
model = SentenceLevelModel()
model.to(device)

SentenceLevelModel(
  (rnn): LSTM(64, 20)
  (classifier): Linear(in_features=20, out_features=2, bias=True)
)

In [55]:
out = model(sent_state,result)

In [56]:
out

tensor([[ 0.4173, -0.2529],
        [ 0.5290, -0.1796],
        [ 0.9462, -0.4894],
        ...,
        [ 1.1264, -0.4452],
        [ 1.1490, -0.6039],
        [ 0.5990,  0.3318]], device='cuda:0', grad_fn=<AddBackward0>)

In [57]:
result

tensor([[ 0.3698, -0.6745],
        [ 0.3115, -0.6124],
        [ 1.1771, -1.3010],
        ...,
        [ 1.4368, -1.6777],
        [ 1.4608, -2.0875],
        [ 0.1937, -0.1417]], device='cuda:0')